# CAFG Algo Platform (Competition 2016) - Tutorial

This document assumes some familiarity with the Jupyter/Notebook environment (please refer to the Intro documentation if you need some help with it) and some experience in Python.

We will focus on the aspects of the platform that are specific to Algo Trading.

## Market Data

For the CAFG competition, we have restricted our platform to the backtest of trading strategies. You will build a trading program then evaluate its performance based on historical data. During the development phase, we are providing 6 months of historical data (from Jan to Jun 2016). You have access to the market data of every future and option of the Hong Kong Hang Seng and H-Share indexes. Since both have a mini version, there are in total 4 underlying products: HHI, HSI, MHI and MCH.

Considering the quantity of options and in order to make computation times reasonable, we sampled the data to a one second interval. There are at most one change of bid/ask and one change of last price for a given contract per second. 

Every quote has a price and a quantity. The Stock Exchange simulator uses the quantity to limit the quantity matched. For instance, if there are 4 contracts for sale and your algo tries to buy 5 contracts, only 4 contracts will be bought.

The system samples the last Bid/ask value in a 1-second window. It stores the total quantity traded and the last snapshot traded price. 

## Structure of an algo

Fundamentally, your algo is a program listening to market data changes and submitting buy/sell orders as a result. Typically, the structure of your algo follows this design:

- Choose a set of contracts
- Whenever the market data of one of these contract changes, receive a notification
- On a notification, do some calculation and update internal state variables
- Use these variables as a signal to trade
- Trade if a signal is raised unless the position has reached a limit
- Monitor the execution of your orders
- Look at the current portfolio and calculate your profit/loss
- Stop trading, i.e. close your position if your P/L goes out of a particular range (profit taking or stop loss)

Of course, these are guidelines and your algo is free to follow another model.

In this tutorial document, we will make a simple algo that follows these steps.

The algo trades the HSI future Jan 2016. It only uses the last traded price and ignores bid/ask. It computes two moving averages, one over the last 60 samples and the other over the last 3000 samples. Every day, it will reset and not do anything during the first 3000 samples while it accumulates data. Then, whenever the short average goes over the long average by more than 0.1%, it opens a buy position of 1 HSI contract. If the opposite happens and the short average goes below the long average by more than 0.1%, it opens a sell position instead. The position should not exceed 5 contracts (buy or sell). If the profit exceeds 10% during the day, it closes the position. If the loss exceeds 5% it also closes the position. Regardless of the profit or loss, it closes the position at the end of the day.

We can see that we are going to need to compute two moving averages. It will be useful to come up with a moving average class. 

## Environment

The CAFG platform API exposes an object called the `BackTestEnvironment`. It groups several services together and serves as a top level entry point. To create it, we first need to define the `AccountSettings` which mainly sets our capital available.

Assuming that we have 1 million in cash, we can create our environment:

In [97]:
from cafg import *

# ts stands for trading strategy
ts = {}

ts["account_settings"] = AccountSettings(1000000, allow_short_sell = True)
ts["environment"] = BacktestEnvironment("20160101", ts["account_settings"])

We allow short selling (selling without holding a product) because we have a margin account. The date 01/01/2016 denotes the beginning of the range of our backtest.

In [98]:
def env_init():
    ts = {"capital": 1000000}

    ts["account_settings"] = AccountSettings(ts["capital"], allow_short_sell = True)
    ts["environment"] = BacktestEnvironment("20160101", ts["account_settings"])    

    ts["md"] = ts["environment"].md
    ts["exch"] = ts["environment"].exch
    ts["account"] = ts["environment"].account
    return ts
                       
ts = env_init()

## Developer Note
> We choose to store data in a dictionary instead of using member variables. Both options are equally fine and the object oriented approach may even be superior in some cases. However, for this tutorial we want to keep the code repetition to a minimum yet introduce concepts incrementally. Since class members cannot be defined individually, we would have had to show the complete class whenever we make any change to its content. We believe it make them harder to see.

These variables are properties of the environment and are extracted for convenience. `md` is our source of market data, `exch` is an object that handles our trading orders and `account` tracks the current position and profit/loss. They are automatically updated whenever your algo submits orders.

Now we need to subscribe to market data. The model is "event-driven". It means that your program should be designed to respond to *events* that are coming from the framework. The events are of various nature. Most of them will be related to market data but you can also receive events when the stock exchange fills an order or at the end of the day.

Instruments (here contracts and securities) are given a unique number. Therefore, we should find what is the number for the HSI Future Jan 2016.

The CAFG platform supports multiple exchanges. For the competition, the data is in exchange #3.

In [99]:
catalog = ExchangeCatalog(3)
type(catalog)

cafg.cmkd.ExchangeCatalog

In [100]:
c = catalog.list()
# First 10 products
c[0:10]

[(ID:17443 CC:HSI    F:O EC:1609 ED:20160929, K:20200.0 CP:P PF:50),
 (ID:10431 CC:HSI    F:O EC:1604 ED:20160428, K:14400.0 CP:C PF:50),
 (ID:15990 CC:HHI    F:O EC:1606 ED:20160629, K:15800.0 CP:C PF:50),
 (ID:7357 CC:HSI    F:O EC:1603 ED:20160330, K:19000.0 CP:P PF:50),
 (ID:19066 CC:MHI    F:O EC:1608 ED:20160830, K:16600.0 CP:C PF:10),
 (ID:17612 CC:HSI    F:O EC:1612 ED:20161229, K:19800.0 CP:P PF:50),
 (ID:1969 CC:MHI    F:O EC:1601 ED:20160128, K:27600.0 CP:P PF:10),
 (ID:16336 CC:HHI    F:O EC:1609 ED:20160929, K:5900.0 CP:C PF:50),
 (ID:14715 CC:MHI    F:O EC:1605 ED:20160530, K:12600.0 CP:P PF:10),
 (ID:17093 CC:HSI    F:O EC:1607 ED:20160728, K:17400.0 CP:C PF:50)]

This gets you a pretty long list of instruments. We displayed the first 10 entries. The catalog includes products that we have filtered out of the market data. So even if you have the product code for A50 options, you will not see any quotes for them. If we want to select the products based on the HSI index, we can use a filter on the class code as follows. Note that we must use `strip()` because the class code is padded to 6 characters.

In [101]:
hsi_products = [x for x in catalog.list() if x.class_code.strip() == "HSI"]
len(hsi_products)

2182

That still leaves us with about 2000 products though. Let's limit ourselves to futures only.

In [102]:
[x for x in catalog.list() if x.class_code.strip() == "HSI" and x.fut_opt == 'F']

[(ID:16829 CC:HSI    F:F EC:1606 ED:20160629, K:0.0 CP:  PF:50),
 (ID:13497 CC:HSI    F:F EC:1605 ED:20160530, K:0.0 CP:  PF:50),
 (ID:17036 CC:HSI    F:F EC:1607 ED:20160728, K:0.0 CP:  PF:50),
 (ID:7289 CC:HSI    F:F EC:1603 ED:20160330, K:0.0 CP:  PF:50),
 (ID:751 CC:HSI    F:F EC:1601 ED:20160128, K:0.0 CP:  PF:50),
 (ID:17353 CC:HSI    F:F EC:1609 ED:20160929, K:0.0 CP:  PF:50),
 (ID:18968 CC:HSI    F:F EC:1608 ED:20160830, K:0.0 CP:  PF:50),
 (ID:4122 CC:HSI    F:F EC:1602 ED:20160226, K:0.0 CP:  PF:50),
 (ID:10410 CC:HSI    F:F EC:1604 ED:20160428, K:0.0 CP:  PF:50),
 (ID:17526 CC:HSI    F:F EC:1612 ED:20161229, K:0.0 CP:  PF:50)]

Now we are down to a handful of products. The only difference between these contracts are their maturity.
The first contract has expiration code 1606 which is 06/2016. Precisely, the expiration is 20160629 or 29/06/2016. K is the strike. Since futures have no strike, it is 0. CP stands for call or put. It's blank for a future. PF is the price multiplication factor. Every change of 1\$ of the HSI means a profit or loss of 50\$ per contract.

We can see that our contract has ID = 751. Programmatically, we can get it like this:

In [103]:
id = [x for x in catalog.list() if x.class_code.strip() == "HSI" and x.fut_opt == 'F' and x.expiration_code == '1601'][0].orderbook_id
print(id)

751


We are going to collect the last traded price for the day of 04/01/2016 (first trading day of the year) and put it in an array.

In [104]:
import functools
ts["last_prices"] = []
def on_quote(ts, q):
    if q.status == QuoteStatus.Trade:
        ts["last_prices"].append(q.last.px)
    
with ts["md"].quotes.subscribe(functools.partial(on_quote, ts)):
    ts["md"].run("20160104", "20160105", [Product(3, id)])

We start with an empty array `last_prices` and define a function (a callback) that the framework will call whenever it gets a market quote from the database. The callback appends the last price at the end of our array. Then we hook up the callback to the market data quotes event handler using the `subscribe` function. Subscribe returns a value that we should `close` when we are no longer need the subscription. Using the Python `with` syntax, we can automatically close the subscription when we exit the block under `with`. Finally, we ask the framework to retrieve the market data between 04/01/2014 and 05/01/2016, and call our event handlers. At the end of evaluation, last_prices has about ~18000 values.

In [105]:
len(ts["last_prices"])

17827

## Moving average

Our trading algo uses two averages based on a sliding window over the last N values of the prices. We collected them in a vector but we don't need to keep them all. For the moving average of length N, we need to calculate the average of the last N values. A brute force method is to keep a buffer and accumulate values until it fills up. At that point, the we can calculate the first output. From that point, when we get a new value, we discard the oldest values and slide everything over to make room for the new entry.

It is a correct method though we can do better. First of all, we don't have to physically move the items over. We can keep the index of the last entry and logically move the tail of the buffer by incrementing the index.

In [106]:
N = 10
buffer = []
tail_index = 0

def add_to_buffer(v):
    global tail_index
    if len(buffer) < N:
        buffer.append(v)
    else:
        buffer[tail_index] = v
    tail_index += 1
    if tail_index == N:
        tail_index = 0

In [107]:
for i in range(15):
    add_to_buffer(i)
    print(buffer)

[0]
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 5, 6, 7, 8, 9]



Turning this into a class, we get the following version.

In [108]:
class MovingAverage:
    def __init__(self, N):
        self.N = N
        self.buffer = []
        self.tail_index = 0
        
    def add(self, v):
        if len(self.buffer) < self.N:
            self.buffer.append(v)
        else:
            self.buffer[self.tail_index] = v
        self.tail_index += 1
        if self.tail_index == N:
            self.tail_index = 0
            
    def get(self):
        return sum(self.buffer)/self.N

MA = MovingAverage(10)
for i in range(15):
    MA.add(i)
    print(MA.buffer, MA.get())

[0] 0.0
[0, 1] 0.1
[0, 1, 2] 0.3
[0, 1, 2, 3] 0.6
[0, 1, 2, 3, 4] 1.0
[0, 1, 2, 3, 4, 5] 1.5
[0, 1, 2, 3, 4, 5, 6] 2.1
[0, 1, 2, 3, 4, 5, 6, 7] 2.8
[0, 1, 2, 3, 4, 5, 6, 7, 8] 3.6
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 4.5
[10, 1, 2, 3, 4, 5, 6, 7, 8, 9] 5.5
[10, 11, 2, 3, 4, 5, 6, 7, 8, 9] 6.5
[10, 11, 12, 3, 4, 5, 6, 7, 8, 9] 7.5
[10, 11, 12, 13, 4, 5, 6, 7, 8, 9] 8.5
[10, 11, 12, 13, 14, 5, 6, 7, 8, 9] 9.5


Here, we are directly using the buffer to compute the average. However, when we notice that the summation term only changes by (newest value - oldest value), we can revise a more performant version.

In [109]:
class MovingAverage:
    def __init__(self, N):
        self.N = N
        self.buffer = []
        self.tail_index = 0
        self.total = 0
        
    def add(self, v):
        if len(self.buffer) < self.N:
            self.buffer.append(v)
            self.total += v
        else:
            oldest_value = self.buffer[self.tail_index]
            self.buffer[self.tail_index] = v
            self.total += v - oldest_value
        self.tail_index += 1
        if self.tail_index == self.N:
            self.tail_index = 0
            
    def is_ready(self):
        return len(self.buffer) == self.N
            
    def get(self):
        return self.total/self.N

MA = MovingAverage(10)
for i in range(15):
    MA.add(i)
    print(MA.buffer, MA.get())

[0] 0.0
[0, 1] 0.1
[0, 1, 2] 0.3
[0, 1, 2, 3] 0.6
[0, 1, 2, 3, 4] 1.0
[0, 1, 2, 3, 4, 5] 1.5
[0, 1, 2, 3, 4, 5, 6] 2.1
[0, 1, 2, 3, 4, 5, 6, 7] 2.8
[0, 1, 2, 3, 4, 5, 6, 7, 8] 3.6
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] 4.5
[10, 1, 2, 3, 4, 5, 6, 7, 8, 9] 5.5
[10, 11, 2, 3, 4, 5, 6, 7, 8, 9] 6.5
[10, 11, 12, 3, 4, 5, 6, 7, 8, 9] 7.5
[10, 11, 12, 13, 4, 5, 6, 7, 8, 9] 8.5
[10, 11, 12, 13, 14, 5, 6, 7, 8, 9] 9.5


This version calculates the moving average in constant time instead of linear time over the window size.

## Trading logic

With the help of the Moving Average class, we can express the trading logic. As a reminder, it is:

> When the short average goes over the long average by more than 0.1%, we open a buy position of 1 HSI contract. If the opposite happens and the short average goes below the long average by more than 0.1%, we open a sell position instead.

In order to implement this logic, we are going to keep the relative position of the short average versus the long average. With a range of +/- 0.1% around the current value of the long average, the short average is either above, below or inside.

If it is inside the range, we will not do anything but when it goes from inside the range to either above or below, we will *try* to trade (if it doesn't break other rules).

In [110]:
def init():
    ts = env_init()
    ts["short_ma"] = MovingAverage(60)
    ts["long_ma"] = MovingAverage(3000)
    ts["signal"] = None
    return ts

def on_quote(ts, q):
    if q.status == QuoteStatus.Trade:
        ts["short_ma"].add(q.last.px)
        ts["long_ma"].add(q.last.px)
        algo_logic(ts)

def algo_logic(ts):
    # Wait until our moving averages have enough data
    if not ts["long_ma"].is_ready():
        return

    # The signal indicates where the short MA is relative to the long MA
    if ts["short_ma"].get() > 1.001 * ts["long_ma"].get():
        signal = 1
    elif ts["short_ma"].get() < 0.999 * ts["long_ma"].get():
        signal = -1
    else:
        signal = 0

    # Act if the signal changed
    if ts["signal"] is not None and ts["signal"] != signal:
        if signal == 1:
            try_trade(ts, 1)
        elif signal == -1:
            try_trade(ts, -1)
    ts["signal"] = signal

n_trades = 0
def try_trade(ts, quantity):
    global n_trades
    # Placeholder
    n_trades += 1

When we run it through the data of 04/01/2016, it would have tried to trade a few times during the day. 

In [111]:
def run():
    ts = init()
    with ts["md"].quotes.subscribe(functools.partial(on_quote, ts)):
        ts["md"].run("20160104", "20160105", [Product(3, id)])

run()
print(n_trades)

12


## Checking the position before trading

Our algo has a limit on the open position. Namely,

> The position should not exceed 5 contracts (buy or sell) and the program should look at the open position before trading

In [112]:
def init():
    ts = env_init()
    ts["short_ma"] = MovingAverage(60)
    ts["long_ma"] = MovingAverage(3000)
    ts["signal"] = None
    ts["order_id"] = 1
    return ts

Let's use a logger that writes to a file. Dumping large data to the console slows down the web browser too much. We can open the file in another tab.

In [113]:
import logging
import os
path = os.environ['HOME'] + '/algo.log'
lh = logging.FileHandler(path)
logger = logging.getLogger()
logger.addHandler(lh)
logger.setLevel(logging.DEBUG)

In [114]:
def display_exec(e):
    logger.info(e)

def try_trade(ts, quantity):
    position = ts["account"].get_position(3, id)
    if position is None or abs(position.quantity + quantity) <= 5:
        side = Side.Buy if quantity > 0 else Side.Sell
        q = abs(quantity)
        logger.info("Trading %d" % quantity)
        ts["exch"].new_order(0, Order(0, ts["order_id"], TimeInForce.ImmediateOrCancel, OrderType.Market, 3, id, 
                                      side, 0.0, q, "", 0, 0))
        ts["order_id"] += 1

def run_range(start, end):
    ts = init()
    with ts["md"].quotes.subscribe(functools.partial(on_quote, ts)), ts["exch"].executions.subscribe(display_exec):
        ts["md"].run(start, end, [Product(3, id)])
    return ts
        
def run(): 
    return run_range("20160104", "20160109")

ts = run()

In [115]:
print(ts["account"].get_position(3, id))

(CODE:751 RPNL:-48217.866666667396 Q:2 AP:20495.821333333333 MP:20435.0 C:1380.0 P:20435.0 PF:50.0)


Tip: One of the great advantages of using a notebook is that the variables are "sticky". They keep their value from one cell to another and also between evaluation of the cells. It leads to a dynamic and interactive workflow that has few interruptions due to technical details. Contrary to a C++ framework, with the CAFG platform you don't have a compile/run cycle. 

However, this persistence comes with a drawback: your algo state is not reset. It is one of the reasons why we chose to keep the algo state in a dictionary. Since we create a new instance every time we run, we know that nothing will leak from one run to the next. If you use global variables and run into trouble with the state of the account and subscriptions, the easiest way start anew is to restart the kernel and rerun all the cells. There is an option on the menu bar for it: `Kernel/Restart and Run All`.

## Performance

Once we have executed some trades, we can check the performance of our strategy using the account statistics.

In [116]:
ts["account"].get_trades()

[(TS:1451875592000000000 OID:1 ID:2 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21398 RS:),
 (TS:1451876438000000000 OID:2 ID:4 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21367 RS:),
 (TS:1451878685000000000 OID:3 ID:6 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21342 RS:),
 (TS:1451879802000000000 OID:4 ID:8 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21329 RS:),
 (TS:1451883625000000000 OID:5 ID:10 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21314 RS:),
 (TS:1451889097000000000 OID:6 ID:12 CODE:751 OS:OrderStatus.Filled SIDE:Side.Buy CQ:1 EQ:1 EP:21277 RS:),
 (TS:1451889962000000000 OID:7 ID:14 CODE:751 OS:OrderStatus.Filled SIDE:Side.Buy CQ:1 EQ:1 EP:21272 RS:),
 (TS:1451891838000000000 OID:8 ID:16 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21251 RS:),
 (TS:1451892171000000000 OID:9 ID:18 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21254 RS:),
 (TS:1451957511000000000 OID:10 ID

In [117]:
ts["account"].get_stats()

,day_end_capital,day_end_exposure,day_end_value,day_pnl,day_start_capital,day_start_value,max_drawdown,max_long,max_profit,max_short,realized_pnl
timestamp,,,,,,,,,,,
2016-01-04,1056915.0,5278250.0,56915.0,56915.0,1000000.0,0.0,765.0,0.0,56915.0,5330250.0,7415.000000
2016-01-05,1051005.0,3178200.0,51005.0,-5910.0,1056915.0,56915.0,65515.0,3178350.0,17760.0,5328250.0,48755.000000
2016-01-06,1012720.0,2087400.0,12720.0,-38285.0,1051005.0,51005.0,40085.0,5263500.0,9300.0,2102200.0,23065.000000
2016-01-07,949755.0,3056850.0,-50245.0,-62965.0,1012720.0,12720.0,77215.0,5114000.0,0.0,0.0,-50665.000000
2016-01-08,944320.0,2043500.0,-55680.0,-5435.0,949755.0,-50245.0,15930.0,5142250.0,45585.0,0.0,-49597.866667


We get a table summaries the daily activities. Since we only ran for one week, we get five rows.

## Stop Loss / Profit Taking

> If the profit exceeds 10% during the day, we close the position. If the loss exceeds 5% we also close the position.

To implement this rule, we need to keep track of the daily P&L.

We will use the `total` method on the account which returns the current total asset value of the portfolio (marked to market) excluding the starting capital. Obviously, the portfolio value will change without us needing to trade as long as we have a position and there is some market activity. Therefore, we need to check the daily P&L on the `on_quote` callback. Profits and Losses are calculated relatively to a `base_value` that we reset when we close the position.

In [118]:
def init():
    ts = env_init()
    ts["short_ma"] = MovingAverage(60)
    ts["long_ma"] = MovingAverage(3000)
    ts["signal"] = None
    ts["order_id"] = 1
    ts["base_value"] = ts["capital"]
    return ts

def on_quote(ts, q):
    if q.status == QuoteStatus.Trade:
        ts["short_ma"].add(q.last.px)
        ts["long_ma"].add(q.last.px)
        algo_logic(ts)
    check_pnl(ts)

def check_pnl(ts):
    av = ts["account"].total()
    pnl = av / (av + ts["base_value"]) * 100.0
    if pnl > 10:
        close_position(ts)
    elif pnl < -5:
        close_position(ts)

def close_position(ts):
    q = ts["account"].get_position(3, id).quantity
    side = Side.Sell if q > 0 else Side.Buy
    ts["exch"].new_order(0, Order(0, ts["order_id"], TimeInForce.ImmediateOrCancel, OrderType.Market, 3, id, 
                                  side, 0.0, abs(q), "", 0, 0))
    ts["order_id"] += 1
    ts["base_value"] += ts["account"].total()
    
ts = run()

Note: The position may not be fully closed if the quantity available at the market is lower than what we need. For simplicity, we ignore this case.

In [119]:
ts["account"].get_stats()

,day_end_capital,day_end_exposure,day_end_value,day_pnl,day_start_capital,day_start_value,max_drawdown,max_long,max_profit,max_short,realized_pnl
timestamp,,,,,,,,,,,
2016-01-04,1056915.0,5278250.0,56915.0,56915.0,1000000.0,0.0,765.0,0.0,56915.0,5330250.0,7415.0
2016-01-05,1051005.0,3178200.0,51005.0,-5910.0,1056915.0,56915.0,65515.0,3178350.0,17760.0,5328250.0,48755.0
2016-01-06,1012720.0,2087400.0,12720.0,-38285.0,1051005.0,51005.0,40085.0,5263500.0,9300.0,2102200.0,23065.0
2016-01-07,948360.0,1016750.0,-51640.0,-64360.0,1012720.0,12720.0,65510.0,4096200.0,0.0,3057450.0,-52140.0
2016-01-08,924885.0,1019700.0,-75115.0,-23475.0,948360.0,-51640.0,31415.0,5135250.0,4920.0,2054500.0,-80165.0


In [120]:
print(ts["account"].get_position(3, id))

(CODE:751 RPNL:-78650.00000000017 Q:-1 AP:20495.0 MP:20394.0 C:1515.0 P:20394.0 PF:50.0)


## Closing the position at the end of the day

The last thing we need to do for our strategy is closing the position at the end of the day. We will listen to market close events and simply call `close_position`. In a real trading environment, we can't do this because when we get the market close event the market is obviously closed and we can't trade anymore. In production, we would listen to the event announcing a market closure. 

In [121]:
def on_quote(ts, q):
    if q.status == QuoteStatus.Trade:
        ts["short_ma"].add(q.last.px)
        ts["long_ma"].add(q.last.px)
        algo_logic(ts)
    elif q.status == QuoteStatus.Close:
        close_position(ts)
    check_pnl(ts)
    
ts = run()
ts["account"].get_stats()

,day_end_capital,day_end_exposure,day_end_value,day_pnl,day_start_capital,day_start_value,max_drawdown,max_long,max_profit,max_short,realized_pnl
timestamp,,,,,,,,,,,
2016-01-04,1054200.0,4222600.0,54200.0,54200.0,1000000.0,0.0,765.0,0.0,56915.0,5330250.0,14600.000000
2016-01-05,1049010.0,2118800.0,49010.0,-5190.0,1054200.0,54200.0,62380.0,3178350.0,20895.0,5328250.0,47510.000000
2016-01-06,1018395.0,1043700.0,18395.0,-30615.0,1049010.0,49010.0,31350.0,5259000.0,8915.0,3153300.0,23842.500000
2016-01-07,975650.0,1018950.0,-24350.0,-42745.0,1018395.0,18395.0,52580.0,5114000.0,0.0,1021050.0,-24490.000000
2016-01-08,954870.0,1021750.0,-45130.0,-20780.0,975650.0,-24350.0,22315.0,5138000.0,21485.0,0.0,-41764.666667


# Full Simulation

Running over one month of data

In [122]:
def run_all():
    return run_range("20160101", "20160201")

In [123]:
ts = run_all()
ts["account"].get_stats()

,day_end_capital,day_end_exposure,day_end_value,day_pnl,day_start_capital,day_start_value,max_drawdown,max_long,max_profit,max_short,realized_pnl
timestamp,,,,,,,,,,,
2016-01-04,1054200.0,4222600.0,54200.0,54200.0,1000000.0,0.0,765.0,0.0,56915.0,5330250.0,14600.000000
2016-01-05,1049010.0,2118800.0,49010.0,-5190.0,1054200.0,54200.0,62380.0,3178350.0,20895.0,5328250.0,47510.000000
2016-01-06,1018395.0,1043700.0,18395.0,-30615.0,1049010.0,49010.0,31350.0,5259000.0,8915.0,3153300.0,23842.500000
2016-01-07,975650.0,1018950.0,-24350.0,-42745.0,1018395.0,18395.0,52580.0,5114000.0,0.0,1021050.0,-24490.000000
2016-01-08,954870.0,1021750.0,-45130.0,-20780.0,975650.0,-24350.0,22315.0,5138000.0,21485.0,0.0,-41764.666667
2016-01-11,910310.0,0.0,-89690.0,-44560.0,954870.0,-45130.0,47230.0,2006600.0,0.0,2988450.0,-89690.000000
2016-01-12,972750.0,3936600.0,-27250.0,62440.0,910310.0,-89690.0,8590.0,0.0,69255.0,5000750.0,-78778.000000
2016-01-13,878585.0,1003450.0,-121415.0,-94165.0,972750.0,-27250.0,105435.0,3031950.0,0.0,5043750.0,-126740.000000
2016-01-14,868300.0,0.0,-131700.0,-10285.0,878585.0,-121415.0,35120.0,4905750.0,17785.0,0.0,-131700.000000


Unfortunately, we see that after a month the strategy is performing poorly and we have a net loss.

# Plots

In order to better understand the P&L profile, we can plot a graph. First we need to collect the values into a time series. The `on_quote` function adds a data point to the chart when it receives a trade. Technically, the P/L moves whenever the bid/ask changes but since we are working with very liquid instruments, we can use trades instead. Timestamps are given in nano seconds. The chart tool uses millisecond units, so the timestamps are divided by 1 000 000. Points are (X = timestamp, Y = total value of portfolio)

In [124]:
import sys
sys.path.append('../helpers')

from plotdata import PlotData
import datetime
import pytz

pnl_plot = []

def on_quote(ts, q):
    if q.status == QuoteStatus.Trade:
        dt = q.timestamp/1000000
        av = ts["account"].total()
        pnl_plot.append([dt, av])
        ts["short_ma"].add(q.last.px)
        ts["long_ma"].add(q.last.px)
        algo_logic(ts)
    elif q.status == QuoteStatus.Close:
        close_position(ts)
    check_pnl(ts)
    
ts = run()

Then we use the Highstock charting tool to create an interactive plot. By default, timestamps are in UTC but we'd rather use local time. By setting the global option `useUTC` to false, we can have Highstock converts t

In [125]:
from highcharts import Highstock
H = Highstock()

options = {
    'rangeSelector' : {
            'selected' : 1
    },
    'title' : {
        'text' : 'P&L'
    },
}
H.setOptions['global'].update_dict(useUTC=False)
H.set_dict_options(options)
H.add_data_set(pnl_plot, 'spline', 'P&L', marker={'enabled': False}) 

H

## Reporting Trades

In [126]:
ts["account"].get_trades()

[(TS:1451875592000000000 OID:1 ID:2 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21398 RS:),
 (TS:1451876438000000000 OID:2 ID:4 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21367 RS:),
 (TS:1451878685000000000 OID:3 ID:6 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21342 RS:),
 (TS:1451879802000000000 OID:4 ID:8 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21329 RS:),
 (TS:1451883625000000000 OID:5 ID:10 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21314 RS:),
 (TS:1451889097000000000 OID:6 ID:12 CODE:751 OS:OrderStatus.Filled SIDE:Side.Buy CQ:1 EQ:1 EP:21277 RS:),
 (TS:1451889962000000000 OID:7 ID:14 CODE:751 OS:OrderStatus.Filled SIDE:Side.Buy CQ:1 EQ:1 EP:21272 RS:),
 (TS:1451891838000000000 OID:8 ID:16 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21251 RS:),
 (TS:1451892171000000000 OID:9 ID:18 CODE:751 OS:OrderStatus.Filled SIDE:Side.Sell CQ:1 EQ:1 EP:21254 RS:),
 (TS:1451896114000000000 OID:10 ID

## Sharpe Ratio

In [127]:
stats = ts["account"].get_stats()

The Sharpe ratio is the average return earned in excess of the risk-free rate per unit of volatility or total risk. If we ignore the risk-free rate, the formula becomes $$ \frac{\overline{R}}{\sqrt{\sigma_R}} $$ where $R$ is the return of the investment or PnL.

In [128]:
dpnl = stats["day_pnl"]
sharpe_ratio = dpnl.mean() / dpnl.std()
sharpe_ratio

-0.23799124903695962

# Conclusion
This tutorial gave a short introduction to the CAFG trading platform backtesting functionality. Going forward, we recommend several resources:

- The API and developer guide,
- The documentation of Numpy, Scipy and Pandas
- Tips & the cookbook documents,
- and of course, any trading strategy book

Good luck!